# Input file requirements:
`./CT-data.dat`

In [ ]:
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as pp
import pandas as pd

from matplotlib import gridspec
from MDAnalysis.analysis.rms import RMSD
from scipy import stats
from IPython.display import Markdown


%matplotlib notebook

In [ ]:
def get_density(vals, *args, **kwargs):
    kernel = stats.gaussian_kde(vals, *args, **kwargs)

    x = np.linspace(min(vals), max(vals), 1000)
    y = kernel(x)

    return x, y

In [ ]:
df = pd.read_csv('CT-data.dat', sep='\s+')
SimTime = 40

df['Time'] = np.arange(0, SimTime, SimTime/df.shape[0])


In [ ]:

fig = pp.figure()
fig.set_size_inches(8,5)
gs = gridspec.GridSpec(1, 3)
gs.update(hspace=0.01, wspace=0.01)



ax_left = fig.add_subplot(gs[0, 0:2])
ticks = ax_left.get_xticks

ax_right = fig.add_subplot(gs[0, 2], sharey=ax_left)


ax_left.set_xlabel(r"Time (ns)", fontsize=16)

ax_right.yaxis.set_visible(False)
ax_right.xaxis.set_visible(False)

'''
ax_left.text(
    0.5, 0.7, sel[0], 
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=ax_left.transAxes,
    fontsize=14
)
'''
df.plot(
    x='Time',
    ax=ax_left
)

for pol in df:
    if pol == 'Time': continue
    x, y = get_density(df[pol], bw_method=0.1)
    
    ax_right.plot(y, x, label=pol)


ax_right.set_xlim(left=0)
ax_left.set_xlim(min(df.Time), max(df.Time)) 

ax_right.legend()
fig.text(
    0.04, 0.5, r"Charge transfer [e]", 
    va='center', rotation='vertical', fontsize=16
)

fig.tight_layout()
pp.show()